<a href="https://colab.research.google.com/github/Raphael-Oliveira-01/Projetos-de-An-lises-financeiras-do-Brasil/blob/main/motor_credito_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -*- coding: utf-8 -*-

"""
PROJETO 1: SIMULADOR DE MOTOR DE ANÁLISE DE CRÉDITO

Este script simula um motor de análise de crédito, aplicando um conjunto
de regras de negócio (política de crédito) a uma base de dados fictícia de
solicitantes para determinar a aprovação, negação ou envio para análise manual.

Tecnologias: Python, Pandas, NumPy
"""
!pip install openpyxl
import pandas as pd
import numpy as np
from datetime import datetime

def gerar_base_solicitantes(num_solicitantes: int = 1000) -> pd.DataFrame:
    """
    Cria uma base de dados fictícia de solicitantes de crédito.

    Args:
        num_solicitantes (int): O número de registros de clientes a serem gerados.

    Returns:
        pd.DataFrame: DataFrame com os dados dos solicitantes.
    """
    print(f"Gerando base de dados com {num_solicitantes} solicitantes...")
    np.random.seed(42)  # Para reprodutibilidade dos resultados

    data = {
        'id_cliente': range(1, num_solicitantes + 1),
        'idade': np.random.randint(18, 75, size=num_solicitantes),
        'renda_mensal': np.random.uniform(1200, 15000, size=num_solicitantes).round(2),
        'score_credito': np.random.randint(0, 1000, size=num_solicitantes)
    }

    df = pd.DataFrame(data)
    print("Base de dados gerada com sucesso.\n")
    return df

def definir_politica_de_credito() -> dict:
    """
    Define e parametriza as regras de negócio (política de crédito).
    Centralizar os parâmetros aqui facilita a manutenção e ajuste das regras.

    Returns:
        dict: Dicionário contendo os pontos de corte para a análise.
    """
    politica = {
        # Regras de Negação Automática (prioridade 1)
        'SCORE_MINIMO_NEGACAO': 300,
        'IDADE_MINIMA': 18,
        'RENDA_MINIMA_NEGACAO': 1500,

        # Regras de Aprovação Automática (prioridade 2)
        'SCORE_MINIMO_APROVACAO': 750,
        'RENDA_MINIMA_APROVACAO': 5000,
        'IDADE_MINIMA_APROVACAO': 25,

        # Regras de aprovação por alto score
        'SCORE_APROVACAO_DIRETA': 950
    }
    return politica

def executar_motor_de_analise(df_solicitantes: pd.DataFrame, politica: dict) -> pd.DataFrame:
    """
    Aplica a lógica de decisão a cada solicitante com base na política de crédito.

    A ordem das regras é *importante*: primeiro verifica-se as condições de negação,
    depois as de aprovação, e o que sobrar vai para análise manual.

    Args:
        df_solicitantes (pd.DataFrame): DataFrame com dados dos clientes.
        politica (dict): Dicionário com os parâmetros da política de crédito.

    Returns:
        pd.DataFrame: DataFrame original com as colunas 'decisao' e 'justificativa'.
    """
    print("Iniciando o processamento pelo motor de análise de crédito...")

    decisoes = []
    justificativas = []

    # Iterar sobre cada linha (solicitante) do DataFrame
    for _, solicitante in df_solicitantes.iterrows():
        idade = solicitante['idade']
        renda = solicitante['renda_mensal']
        score = solicitante['score_credito']

        # --- Etapa 1: Regras de Negação (Red Flags) ---
        if idade < politica['IDADE_MINIMA']:
            decisoes.append('Negado')
            justificativas.append(f"Idade ({idade}) abaixo da mínima permitida ({politica['IDADE_MINIMA']})")

        elif score < politica['SCORE_MINIMO_NEGACAO']:
            decisoes.append('Negado')
            justificativas.append(f"Score de crédito ({score}) abaixo do mínimo para análise ({politica['SCORE_MINIMO_NEGACAO']})")

        elif renda < politica['RENDA_MINIMA_NEGACAO']:
            decisoes.append('Negado')
            justificativas.append(f"Renda mensal (R${renda:.2f}) abaixo da mínima para análise (R${politica['RENDA_MINIMA_NEGACAO']:.2f})")

        # --- Etapa 2: Regras de Aprovação Automática (Green Flags) ---
        elif score >= politica['SCORE_APROVACAO_DIRETA']:
            decisoes.append('Aprovado')
            justificativas.append(f"Score de crédito ({score}) acima do ponto de aprovação direta ({politica['SCORE_APROVACAO_DIRETA']})")

        elif score >= politica['SCORE_MINIMO_APROVACAO'] and \
             renda >= politica['RENDA_MINIMA_APROVACAO'] and \
             idade >= politica['IDADE_MINIMA_APROVACAO']:
            decisoes.append('Aprovado')
            justificativas.append("Cliente atende aos critérios combinados de score, renda e idade para aprovação.")

        # --- Etapa 3: Análise Manual (Yellow Flags) ---
        else:
            decisoes.append('Análise Manual')
            justificativas.append('Não se enquadrou em nenhuma regra automática. Requer revisão manual.')

    # Adicionar os resultados ao DataFrame
    df_solicitantes['decisao'] = decisoes
    df_solicitantes['justificativa_regra'] = justificativas

    print("Análise de crédito concluída.\n")
    return df_solicitantes

def gerar_relatorio_final(df_resultado: pd.DataFrame):
    """
    Exibe um resumo dos resultados e salva o relatório completo em um arquivo Excel.

    Args:
        df_resultado (pd.DataFrame): DataFrame com os resultados da análise.
    """
    print("--- Resumo da Análise de Crédito ---")
    resumo = df_resultado['decisao'].value_counts(normalize=True) * 100
    print(resumo.round(2).to_string())
    print("\n------------------------------------")

    # Gerar o nome do arquivo com timestamp para evitar sobrescrever
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nome_arquivo = f'relatorio_analise_credito_{timestamp}.xlsx'

    try:
        df_resultado.to_excel(nome_arquivo, index=False, engine='openpyxl')
        print(f"Relatório final salvo com sucesso como: '{nome_arquivo}'")
    except Exception as e:
        print(f"Erro ao salvar o relatório: {e}")
        print("Salvando como CSV...")
        nome_arquivo_csv = f'relatorio_analise_credito_{timestamp}.csv'
        df_resultado.to_csv(nome_arquivo_csv, index=False, sep=';', decimal=',')
        print(f"Relatório final salvo com sucesso como: '{nome_arquivo_csv}'")

def main():
    """
    Função principal que comanda a execução do simulador.
    """
    print("=====================================================")
    print("==   Iniciando Simulador de Motor de Crédito       ==")
    print("=====================================================\n")

    # 1. Definição de parâmetros
    NUMERO_DE_SOLICITANTES = 2500

    # 2. Geração da base de dados
    base_solicitantes = gerar_base_solicitantes(NUMERO_DE_SOLICITANTES)

    # 3. Definição da política de crédito
    politica_credito = definir_politica_de_credito()

    # 4. Execução do motor de análise
    base_com_decisao = executar_motor_de_analise(base_solicitantes, politica_credito)

    # 5. Geração do relatório final
    gerar_relatorio_final(base_com_decisao)

    print("\nSimulação finalizada com sucesso!")


if __name__ == "__main__":
    # Garante que o script será executado apenas quando chamado diretamente
    main()

==   Iniciando Simulador de Motor de Crédito       ==

Gerando base de dados com 2500 solicitantes...
Base de dados gerada com sucesso.

Iniciando o processamento pelo motor de análise de crédito...
Análise de crédito concluída.

--- Resumo da Análise de Crédito ---
decisao
Análise Manual    49.84
Negado            32.48
Aprovado          17.68

------------------------------------
Relatório final salvo com sucesso como: 'relatorio_analise_credito_20250819_191301.xlsx'

Simulação finalizada com sucesso!
